In [ ]:
import holoviews as hv
import numpy as np
from holoviews.operation.datashader import datashade
import panel as pn
import pandas as pd
import panel as pn
from holoviews.element.tiles import StamenTerrain

import geoviews as gv
import geoviews.feature as gf
from holoviews import opts
import xarray as xr
import geopandas as gpd
from shapely.geometry import Polygon
import sys

from tqdm import tqdm
import requests


from pathlib import Path
gv.extension("bokeh")


# Geojson

## Setup 

In [ ]:
external_data = Path('../data/external/')
processed_data = Path('../data/processed/')
raw_data = Path('../data/raw/')
interim_data = Path('../data/interim/')

opts.defaults(opts.Polygons(width=800, height=600, toolbar='above', colorbar=True, tools=['hover','tap'], aspect='equal'))

## Lecture données
https://towardsdatascience.com/interactive-geospatial-data-visualization-with-geoviews-in-python-7d5335c8efd1

In [ ]:
france_geo_path = external_data / 'france-geojson-master'
geojson_path = {geo.stem:geo for geo in france_geo_path.rglob('*.parquet')}

regions = geojson_path['regions']
dept = geojson_path['departements']
cont_iris = external_data / 'geojson'/ 'contours-iris.geojson'

if not cont_iris.is_file():
    api_url = 'https://public.opendatasoft.com/explore/dataset/contours-iris/download/?format=geojson&timezone=Europe/Berlin&lang=fr'
    response = requests.get(api_url, stream=True)
    cont_iris.parent.mkdir(exist_ok=True)
    with open(cont_iris, "wb") as handle:
        for data in tqdm(response.iter_content()):
            handle.write(data)


reg_df = gpd.read_parquet(regions.with_suffix('.parquet'))
dept_df = gpd.read_parquet(dept.with_suffix('.parquet'))
cont_iris_df = gpd.read_parquet(cont_iris.with_suffix('.parquet'))

# Cartes
reg_map = gv.Polygons(reg_df, vdims=['nom'])
dept_map = gv.Polygons(dept_df, vdims=['nom','code'])
com_map = gv.Polygons(reg_df, vdims=['nom','code'])

# Extraction de tableau (Indices)
indice_frag_reformated = interim_data / 'Tableau_data_reformat.parquet'
indice_frag = france_geo_path = external_data / 'extract_tableau' / 'Tableau_data.csv'
if not indice_frag_reformated.is_file():
    ifrag_df = pd.read_csv(indice_frag, sep=';', decimal=',')
    ifrag_df.to_parquet(indice_frag.with_suffix('.parquet'))
    ifrag_df_pivot = ifrag_df.pivot(index=['Code Iris','Nom Com','Nom Iris','Classement de SCORE GLOBAL region 1'], columns='Noms de mesures', values=['Valeurs de mesures'])
    ifrag_df_pivot.columns = ifrag_df_pivot.columns.droplevel(0)
    ifrag_df_pivot = ifrag_df_pivot.rename_axis(None, axis=1)
    ifrag_df_pivot.reset_index(inplace=True)
    ifrag_df_pivot.rename({
        'Nom Com':'nom_com',
        'Code Iris':'code_iris',
        'Nom Iris':'nom_iris'}, axis=1, inplace=True
    )
    ifrag_df_pivot.to_parquet(indice_frag_reformated)
else:
    print("{} exist".format(indice_frag_reformated.name))
    ifrag_df = pd.read_parquet(indice_frag.with_suffix('.parquet'))
    ifrag_df_pivot = pd.read_parquet(indice_frag_reformated)

## Indice fragilité communes

In [ ]:
ifrag_df_merged = pd.merge(cont_iris_df,ifrag_df_pivot, on=["code_iris", "nom_com","nom_iris"])
indices_list = list(ifrag_df['Noms de mesures'].unique())
vdims = ['code_iris','nom_com','nom_iris'] + indices_list

# Avec style 😎 

Exemple: https://panel.holoviz.org/gallery/demos/VTKSlicer.html

In [ ]:

css = '''
.panel-widget-box {
  border-radius: 15px;
  border: 1px black solid;

}

.bk-root .bk-input {
	display: inline-block;
	width: 100%;
	flex-grow: 1;
	-webkit-flex-grow: 1;
	min-height: 31px;
	padding: 0 12px;
	background-color: #fff;
	border: 1px solid #ccc;
	border-radius: 15px;
}

'''

In [ ]:
indice = pn.widgets.Select(value=indices_list[0], options=indices_list, name='Indice de fragilité')
nom_com = pn.widgets.AutocompleteInput(
    name='Recherche', options=list(ifrag_df_merged.nom_com.unique()), value='Toulouse',
    placeholder='Toulouse')


@pn.depends(nom_com.param.value, indice.param.value)
def indices_plot(nom_com, indice):
    commune_plot = gv.Polygons(ifrag_df_merged[ifrag_df_merged.nom_com == nom_com], vdims=vdims)
    tiles = StamenTerrain()
    return tiles * commune_plot.opts(color=indice, width=600, height=600, fill_alpha=0.5) 

pn.extension(raw_css=[css])

plot = pn.Row(pn.Column(nom_com, indice), indices_plot) #,css_classes=['panel-widget-box', "bk-root",'bk-input'])
plot.servable()